# Prompt + LLM 

This is a practice for using langchain Prompt. 

## PromptTemplate + LLM 

In [ ]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Provide a short joke about {topic}")
model = ChatOpenAI()
chain = prompt | model

In [23]:
chain.invoke({"topic": "programming"})

AIMessage(content='Why did the programmer go broke?\n\nBecause he lost his domain in a bet!')

In [24]:
chain = prompt | model.bind(stop=["\n"])

In [25]:
chain.invoke({"topic": "programming"})

AIMessage(content='Why did the programmer go broke?')

#### Attaching Function call information

In [26]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [27]:
chain.invoke({"topic": "smartphone"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "setup": "Why did the smartphone bring a ladder?",\n  "punchline": "To reach the cloud!"\n}', 'name': 'joke'}})

## PromptTemplate + LLM + OutputParser

In [28]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

In [29]:
chain.invoke({"topic": "programming"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

### Function Output Parser

In [30]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [31]:
chain.invoke({"topic": "programming"})

{'setup': 'Why do programmers prefer dark mode?',
 'punchline': 'Because light attracts bugs!'}

In [32]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [33]:
chain.invoke({"topic": "programming"})

'Why do programmers prefer dark mode?'

## Simplifying input

Using `RunnablePassthrough` function, pass `topic` as arguement in `RunnableParallel`

In [34]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

map_ = RunnableParallel(topic=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [35]:
chain.invoke("programming")

'Why do programmers prefer dark mode?'

In [36]:
chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [37]:
chain.invoke("programming")

'Why do programmers prefer dark mode?'